In [1]:
import datetime as dt
import spiceypy as sp
import numpy as np
import pandas as pd

In [2]:
sp.furnsh('_kernels/lsk/naif0012.tls')
sp.furnsh('_kernels/spk/de440.bsp')
sp.furnsh('_kernels/pck/pck00010.tpc')

Last time we computed and visualised the movement of the Solar System Barycentre (SSB) with respect to the Sun. A 2-dimensional projection on the Ecliptic plane revealed an impressive behaviour of the SSB. The barycentre's position leaves the Sun on a regular basis, although the Sun contains more than 99% of the Solar System's mass. One can understand that this movement is caused by the other celestial objects like planets, the asteroids and even the dust. But what are the main contributors and can we visualize it?

For our analysis we will beed the same time interval and methods as shown last time. We will use additionally pandas.

Again we choose the time interval starting at the 1st January 2000 and add 10,000 days. We convert the <b>UTC time</b> strings to the <b>Ephemeris time (ET)</b> with the function <i><b>utc2et</b></i> and create a <b>numpy array</b> that contains all the ET time steps.

We want to compute the miscellaneous positions w.r.t. the centre of the Sun for a certain time interval

In [3]:
import datetime as dt
import spiceypy as sp
import numpy as np
import pandas as pd

# We want to compute the miscellaneous positions w.r.t. the centre
# of the Sun for a certain time interval
# First, we set an initial time in UTC.
INIT_TIME_UTC = dt.datetime(year=2000, month=1, day=1, hour=0, minute=0, second=0)

# Add a number of days: you can play with this
DELTA_DAYS=10000
END_TIME_UTC = INIT_TIME_UTC + dt.timedelta(days=DELTA_DAYS)

# Convert the datetime objects now to strings
INIT_TIME_UTC_STR = INIT_TIME_UTC.strftime('%Y-%m-%dT%H:%M:%S')
END_TIME_UTC_STR = END_TIME_UTC.strftime('%Y-%m-%dT%H:%M:%S')

# print the starting and end times
print('Init time in UTC: %s' % INIT_TIME_UTC_STR)
print('End time in UTC: %s\n' % END_TIME_UTC_STR)

# Convert to Ephemeris Time (ET) using the SPICE function utc2et
INIT_TIME_ET = sp.utc2et(INIT_TIME_UTC_STR)
END_TIME_ET = sp.utc2et(END_TIME_UTC_STR)

# Create a numpy array that covers a time interval in delta = 1 day step
TIME_INTERVAL_ET = np.linspace(INIT_TIME_ET, END_TIME_ET, DELTA_DAYS)

Init time in UTC: 2000-01-01T00:00:00
End time in UTC: 2027-05-19T00:00:00



In [4]:
len(TIME_INTERVAL_ET)

10000

How should we proceed now with the analysis of the SSB's movement? Well first of all, we need to compute the trajectory of the barycentre with respect to the Sun. Last time we did this using a for-loop, now we will use pandas DataFrame to store all the necessary results in one place.

SPICE returns the spatial information in km. Again we want to scale it with Syn's radius. We extract this information from the kernel with the function <b><u><i>bodvcd</i></u></b>.

Using <b>km</b> is not intuitive. <b>AU</b> would scale it too severely. Since we compute the Solar System Barycentre (SSB) w.r.t. the Sun and since we expect it to be close to the Sun, we scale the <u>x,y,z component w.r.t. the radius of the Sun</u>. We extract the Sun radii (x,y,z components of the Sun ellipsoid) and use the x component.

In [5]:
# Using km is not intuitive. AU would scale it too severely. 
# Since we compute the Solar System Barycentre (SSB) w.r.t. the 
# Sun and since we expect it to be close to the Sun, we scale 
# the x,y,z component w.r.t. the radius of the Sun. 
# We extract the Sun radii (x,y,z components of the Sun ellipsoid)
# and use the x component.

_, RADII_SUN = sp.bodvcd(bodyid=10, item='RADII', maxn=3)
RADIUS_SUN = RADII_SUN[0]



The results will be stored in a SOLAR_SYSTEM_DF.

In [6]:
# All our parameters, positions etc. shall be stored in a df.
SOLAR_SYSTEM_DF = pd.DataFrame()

# Set the column ET that stores all ETs
SOLAR_SYSTEM_DF.loc[:, 'ET'] = TIME_INTERVAL_ET

# The column UTC transforms all ETs back to UTC format. 
# The function sp.et2datetime is not official part of the SPICE
# There it is et2utc. However the function returns immediately a datetime
# object.

SOLAR_SYSTEM_DF.loc[:, 'UTC'] = SOLAR_SYSTEM_DF['ET'].apply(lambda x: sp.et2datetime(et=x).date())

In [7]:
SOLAR_SYSTEM_DF.head()

ET         UTC
0  -43135.816087  2000-01-01
1   43272.825277  2000-01-02
2  129681.466641  2000-01-03
3  216090.108006  2000-01-04
4  302498.749370  2000-01-05

We add and compute 3 new columns. First POS_SSB_WRT_SUN is computed that contains the position vector of the SSB as seen from the Sun. For this computation we need the ET column and apply row-wise (using <b>.apply()</b>) the SPICE function <b><i><u>spkgps</u></i></b> via a lambda function. <b><i><u>spkgps</u></i></b> requires the target NAIF ID (targ), the ET (et), the reference frame (ref: here it is the Ecliptic reference frame <b>ECLIPJ2000</b>) and the observer's NAIF ID (obs). The ID for the <u>SSB and Sun is 0 and 10</u> respectively.

The computed position vectors are then scaled with the Sun's radius. We apply the row-wise lambda function that divides the arrays by the radius.

Finally the distance between the Sun and the SSB is computed, using the SPICE function <b><i><u>vnorm</u></i></b>. The function compues the length (so-called norm) of a vector and is identical to the numpy function <i>numpy.linalg.norm()</i>.

In [8]:
# Here the position of the SSB, as seen fromthe Sun is computed.
# Since sp.spkgps returns the position and the corresponding light time,
# we add the index [0] to obtain only the position array.

SOLAR_SYSTEM_DF.loc[:, 'POS_SSB_WRT_SUN'] = SOLAR_SYSTEM_DF['ET'].apply(lambda x: sp.spkgps(targ=0, et=x, ref='ECLIPJ2000', obs=10)[0])

# Finally the distance between the Sun and the SSB is computed
SOLAR_SYSTEM_DF.loc[:, 'POS_SSB_WRT_SUN_SCALED'] = SOLAR_SYSTEM_DF['POS_SSB_WRT_SUN'].apply(lambda x: x / RADIUS_SUN)

# Finally the distance between the Sun and the SSB is computed. The length
# (norm) of the vector needs to be determined with the SPICE function
# vnorm(). numpy provides an identical function in numpy.linalg.norm()
SOLAR_SYSTEM_DF.loc[:, 'SSB_WRT_SUN_SCALED_DIST'] = SOLAR_SYSTEM_DF['POS_SSB_WRT_SUN_SCALED'].apply(lambda x: sp.vnorm(x))

In [9]:
SOLAR_SYSTEM_DF.head()

ET         UTC  \
0  -43135.816087  2000-01-01   
1   43272.825277  2000-01-02   
2  129681.466641  2000-01-03   
3  216090.108006  2000-01-04   
4  302498.749370  2000-01-05   

                                     POS_SSB_WRT_SUN  \
0  [1068108.3542452406, 417721.91444219963, -3086...   
1  [1067303.666657607, 418830.0833823817, -30854....   
2  [1066497.7243846857, 419937.0428411623, -30840...   
3  [1065690.5295479323, 421042.79723565804, -3082...   
4  [1064882.0840889374, 422147.350974351, -30812....   

                              POS_SSB_WRT_SUN_SCALED  SSB_WRT_SUN_SCALED_DIST  
0  [1.5346384400075295, 0.6001751644284478, -0.04...                 1.648421  
1  [1.53348227968047, 0.6017673611815829, -0.0443...                 1.647925  
2  [1.5323243166446634, 0.6033578201740838, -0.04...                 1.647429  
3  [1.5311645539481786, 0.6049465477523822, -0.04...                 1.646933  
4  [1.5300029943806572, 0.6065335502505043, -0.04...                 1.646436

Whether it is data science, space science, astronomy or any other scientific field, visualizing and describing data is the first step of a proper analysis.

So let’s visualise the distance between the SSB and the Sun as a function of time. For this purpose we use the module matplotlib, as introduced last time. We use the data of the column SSB_WRT_SUN_SCALED_DIST and plot it against the UTC date-time.